In [ ]:
import glob
import pandas as pd
import numpy as np

# Execution of Emulations

In [ ]:
print("Todo")


# Preparation of Results

In order to get to the results, basically two options exist. The first option is to generate own results by executing the emulations as described above. The second way is to download the original paper results via FTP.

In the case that the results were reproduced, place the results in the `results` folder of the repository.

In order to download the results, execute command below:

__Attention__: About 40GB of disk space are required.

In [ ]:
!../results/download.sh

# Analysis of results

The first step at result analysis is to parse the Wireshark packets and write statistics to handy CSV files. This is done by executing a python script.

__Attention:__ The Python script uses all available CPU cores and could need up to an our to finish calculation.

In [ ]:
!mkdir -p $ANALYSIS_HOME/results/averages
!python3 parsePCAP.py -i $ANALYSIS_HOME/results/final-results -o $ANALYSIS_HOME/results/averages

# Plotting of charts

Once the averages are calculated and available, only the charts need to be plotted. Therefore, use the following command:

In [ ]:
!python3 plotPcapResults.py -i $ANALYSIS_HOME/results/averages -o ./ -s 2x1,3x1,2x2

The resulting charts can be found in the `result-analysis` folder.

# Calculation of overhead

To quatify the percentage of overhead, the following script can be used.

In [ ]:
files = glob.glob("../results/averages/*.csv")

averages = []
for file in files:
    scenario = file.split("/")[-1].split(".")[0]
    df = pd.read_csv(file, delimiter="\t")
    incoming = df[df["in/out"] == "in"]
    outgoing = df[df["in/out"] == "out"]

    sum = np.sum(incoming)
    approach = scenario.split("_")[4]
    size = scenario.split("_")[3]
    clients = scenario.split("_")[1]
    averages.append([ approach, size, int(clients), sum["bytesInterests"], sum["bytesData"], sum["bytesSyncPayload"]])
    
averages = pd.DataFrame(averages, columns=["approach", "size", "#clients", "bytesInterests", "bytesData", "bytesSyncPayload"])
averages["bytesTotal"] = averages["bytesInterests"] + averages["bytesData"]
averages["percentPayload"] = averages["bytesSyncPayload"] / averages["bytesTotal"]
 
averages.sort_values(by=["approach", "size", "#clients"])

# Plotting player movements

One chart in the paper visualized the player movement from the Fortnite Traces [1]. To print this chart, use the following commands.

The resulting figures can be found in the `result-analysis`-folder.

[1] P. Moll, M. Lux, S. Theuermann, & H. Hellwagner. (2018). A Network Traffic and Player Movement Model to Improve Networking for Competitive Online Games. Proceedings of the 15th Annual Workshop on Network and Systems Support for Games. DOI: 10.1109/NetGames.2018.8463390


In [ ]:
!git clone https://github.com/phylib/FortniteTraces FortniteTraces
!python3 plotPlayerMovement.py -i FortniteTraces/GeneratedTraces/g0/ -o ./ -z 1x1,2x1,3x1,2x2